In [1]:
# !pip install unidecode

In [2]:
# !conda install --yes -c conda-forge unidecode

In [3]:
import re, os, sys, requests, time, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import bs4
from bs4 import BeautifulSoup
from unidecode import unidecode
from random import randint

In [4]:
print(f'O notebook está na pasta: {os.getcwd()}')

O notebook está na pasta: C:\Users\viniz\OneDrive\Área de Trabalho\Vinícius\Insper\Matérias\Eletivas\Análise de Texto e Fontes Desestruturadas e Web\Projeto\projeto-fontes-desestruturadas-e-web


In [5]:
url_capitais = 'https://www.todamateria.com.br/capitais-do-brasil/'

In [6]:
uf_capital = {} # Exemplo: {'SP': 'São Paulo', 'RJ', 'Rio de Janeiro'}
resposta = requests.get(url_capitais)
resposta.encoding = 'utf-8'

In [7]:
soup = BeautifulSoup(resposta.text, 'html.parser')

In [8]:
tabela = soup.find('tbody')
linhas = tabela.find_all('tr')
for linha in linhas:
    uf = linha.find_all('td')[2].text
    uf = uf.lower()
    capital = linha.find_all('td')[0].text
    capital = capital.lower()
    capital = capital.split()
    capital = '-'.join(capital)
    capital = unidecode(capital)
    uf_capital[uf] = capital
print(uf_capital)

{'ac': 'rio-branco', 'al': 'maceio', 'ap': 'macapa', 'am': 'manaus', 'ba': 'salvador', 'ce': 'fortaleza', 'df': 'brasilia', 'es': 'vitoria', 'go': 'goiania', 'ma': 'sao-luis', 'mt': 'cuiaba', 'ms': 'campo-grande', 'mg': 'belo-horizonte', 'pa': 'belem', 'pb': 'joao-pessoa', 'pr': 'curitiba', 'pe': 'recife', 'pi': 'teresina', 'rj': 'rio-de-janeiro', 'rn': 'natal', 'rs': 'porto-alegre', 'ro': 'porto-velho', 'rr': 'boa-vista', 'sc': 'florianopolis', 'sp': 'sao-paulo', 'se': 'aracaju', 'to': 'palmas'}


In [9]:
urls = []
for uf, capital in uf_capital.items():
    url = 'https://www.zapimoveis.com.br'
    uri = '/venda/imoveis/'
    uri += f'{uf}+{capital}'
    url += uri
    urls.append(url)
urls

['https://www.zapimoveis.com.br/venda/imoveis/ac+rio-branco',
 'https://www.zapimoveis.com.br/venda/imoveis/al+maceio',
 'https://www.zapimoveis.com.br/venda/imoveis/ap+macapa',
 'https://www.zapimoveis.com.br/venda/imoveis/am+manaus',
 'https://www.zapimoveis.com.br/venda/imoveis/ba+salvador',
 'https://www.zapimoveis.com.br/venda/imoveis/ce+fortaleza',
 'https://www.zapimoveis.com.br/venda/imoveis/df+brasilia',
 'https://www.zapimoveis.com.br/venda/imoveis/es+vitoria',
 'https://www.zapimoveis.com.br/venda/imoveis/go+goiania',
 'https://www.zapimoveis.com.br/venda/imoveis/ma+sao-luis',
 'https://www.zapimoveis.com.br/venda/imoveis/mt+cuiaba',
 'https://www.zapimoveis.com.br/venda/imoveis/ms+campo-grande',
 'https://www.zapimoveis.com.br/venda/imoveis/mg+belo-horizonte',
 'https://www.zapimoveis.com.br/venda/imoveis/pa+belem',
 'https://www.zapimoveis.com.br/venda/imoveis/pb+joao-pessoa',
 'https://www.zapimoveis.com.br/venda/imoveis/pr+curitiba',
 'https://www.zapimoveis.com.br/venda

In [10]:
# ===== APENAS PARA A PRIMEIRA PÁGINA DE CADA CAPITAL POR ENQUANTO =====
'''
ANOTAÇÕES

- Fazer com que o código pesquise no Google o endereço de cada local e
retorne separadamente bairro, número, tipo de logradouro e
nome do logradouro;
OU
- Fazer com que o código pesquise no Google o endereço de cada local e
retorne a latitude e longitude, plotando num mapa ao decorrer do projeto;

- Fazer o Web Crawling de todas as páginas disponíveis para cada cidade
(é esperado que demore, pois cidades como São Paulo possuem milhões de
imóveis listados, talvez seja necessário considerar um limite máximo
como 10 mil imóveis por cidade);

- Não fazer a pesquisa com base nas ordenações disponíveis;

- Fazer com que o código acesse imóvel por imóvel e retone informações
booleanas adicionais do imóvel, por exemplo, se possui elevador (quantidade?),
se possui piscina (quantidade?), se possui academia, se possui espaços
para festas gerais;
''';

In [11]:
posicoes = []
enderecos = []
areas = []
quartos = []
banheiros = []
vagas = []
precos = []
conds = []
iptus = []
descricoes = []
lista_status = []
paginas = []

for url in urls:
    posicao = 1
    pagina = 1
    
    n1, n2, n3 = randint(0, 36), randint(0, 142), randint(0, 35)
    headers = {'user-agent':
               f'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.{n1} (KHTML, like Gecko) Chrome/75.0.3770.{n2} Safari/537.{n3}'
    }
    resposta = requests.get(url, headers=headers)
    resposta.encoding = 'utf-8'
    
    soup = BeautifulSoup(resposta.text, 'html.parser')
    
    lista_prod = soup.find('div', class_='listings__container')
    produtos = lista_prod.find_all('div', class_='card-container js-listing-card')
    for produto in produtos:
        
        # ----- Posição na página
        posicoes.append(posicao)
        posicao += 1
        
        # Endereço
        try:
            endereco = produto.find('h2', attrs={'itemprop': 'address'}).text
            endereco = endereco.strip('\n ')
            enderecos.append(endereco)
        except:
            enderecos.append(None)
        
        # ----- Área
        try:
            area = produto.find('li', class_='feature__item text-small js-areas').text
            area = area.strip('\n ')
            area = area.replace('m²', '')
            areas.append(area)
        except:
            areas.append(None)
        
        # ----- Qtd quartos
        try:
            quarto = produto.find('span', attrs={'itemprop': 'numberOfRooms'}).text
            quarto = quarto.strip('\n ') # Pode ser algo do tipo: '1 - 3'
            quartos.append(quarto)
        except:
            quartos.append(None)
        
        # ----- Qtd banheiros
        try:
            banheiro = produto.find('span', attrs={'itemprop': 'numberOfBathroomsTotal'}).text
            banheiro = banheiro.strip('\n ') # Pode ser algo do tipo: '1 - 3'
            banheiros.append(banheiro)
        except:
            banheiros.append(None)
        
        # ----- Qtd vagas
        try:
            vaga = produto.find('li', class_='feature__item text-small js-parking-spaces').text
            vaga = vaga.strip('\n ') # Pode ser algo do tipo: '1 - 3'
            vagas.append(vaga)
        except:
            vagas.append(None)
        
        # ----- Preço condomínio
        try:
            cond = produto.find('li', class_='card-price__item condominium text-regular').text
            conds.append(''.join(re.findall(r'\d+', cond)))
        except:
            conds.append(None)
            
        # ----- IPTU
        try:
            iptu = produto.find('li', class_='card-price__item iptu text-regular').text
            iptus.append(''.join(re.findall(r'\d+', iptu)))
        except:
            iptus.append(None)
        
        # ----- Decrição
        try:
            descricao = produto.find('span', attrs={'itemprop': 'description'}).text
            descricoes.append(descricao)
        except:
            descricoes.append(None)
        
        # ----- Status
        try:
            status = produto.find('small').text
        except:
            status = 'Comum'
        lista_status.append(status)
        
        # ----- Número da página
        paginas.append(pagina)
        
        # ----- Preço
        preco = produto.find('p', class_='simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left').text
        precos.append(''.join(re.findall(r'\d+', preco)))
        
    pagina += 1

In [12]:
dicionario = {'Posição': posicoes,
              'Endereço': enderecos,
              'Área': areas,
              'Quartos': quartos,
              'Banheiros': banheiros,
              'Vagas': vagas,
              'Condomínio': conds,
              'IPTU': iptus,
              'Descrição': descricoes,
              'Status': lista_status,
              'Página': paginas,
              'Preço': precos}

In [13]:
df = pd.DataFrame(dicionario)
df.head()

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
0,1,"Rodovia BR-364, Portal da Amazonia",492,None,None,None,None,None,O lote/terreno está localizado no bairro Porta...,Super destaque,1,450000
1,2,"Rodovia BR-364, Portal da Amazonia",429,None,None,None,None,None,O lote/terreno está localizado no bairro Porta...,Super destaque,1,360000
2,3,"Rua Severina Maria de Sousa e Silva, Jardim Ma...",99,3,2,2,690,None,O apartamento está localizado no bairro Jardim...,Super destaque,1,570000
3,4,"Rua C, LBA Vila Betel ll",336,2,2,1,None,None,?Casa na Vila Betel ! Vende-se uma casa no co...,Comum,1,185000
4,5,"Rua Abacabeira, Vila da Amizade",243,3,1,2,None,None,A casa está localizado no bairro Vila da Amiza...,Comum,1,380000


In [14]:
cwd = str(os.getcwd())
_, _, files = next(os.walk(cwd))
file_count = len(files)
print(file_count)

df.to_csv(f'data{file_count}.csv')

2


In [15]:
# Tipos de status
df['Status'].value_counts().index

Index(['Comum', 'Destaque', 'Super destaque', 'Em construção', 'Na planta',
       'Pronto para morar'],
      dtype='object')

In [16]:
# Quantidade bruta de imóveis por tipo de status
df['Status'].value_counts()

Comum                331
Destaque             167
Super destaque        81
Em construção         30
Na planta             22
Pronto para morar     17
Name: Status, dtype: int64

In [17]:
# Quantidade relativa de imóveis por tipo de status
df['Status'].value_counts(True).round(4)*100

Comum                51.08
Destaque             25.77
Super destaque       12.50
Em construção         4.63
Na planta             3.40
Pronto para morar     2.62
Name: Status, dtype: float64

In [18]:
super_mask = df['Status'] == 'Super destaque'
super_df = df.loc[super_mask, :]
super_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
0,1,"Rodovia BR-364, Portal da Amazonia",492,None,None,None,None,None,O lote/terreno está localizado no bairro Porta...,Super destaque,1,450000
1,2,"Rodovia BR-364, Portal da Amazonia",429,None,None,None,None,None,O lote/terreno está localizado no bairro Porta...,Super destaque,1,360000
2,3,"Rua Severina Maria de Sousa e Silva, Jardim Ma...",99,3,2,2,690,None,O apartamento está localizado no bairro Jardim...,Super destaque,1,570000
27,4,"Avenida Doutor Antônio Gomes de Barros, Jatiúca",450,4,5,5,2500,None,Linda e confortável cobertura Duplex à Beira M...,Super destaque,1,3100000
28,5,"Rua Ruthe Reis, Poço",57,2,2,1,None,None,Apartamento 02 quartos com suite sala cozinh...,Super destaque,1,370000
...,...,...,...,...,...,...,...,...,...,...,...,...
604,5,"Avenida Conselheiro João Moreira Filho, Coroa ...",78,3,3,1,600,None,Apartamento no 7º andar com vista para o mar n...,Super destaque,1,420000
605,6,"Rua Professor Damião Teles de Menezes, Jabotiana",62,2,1,1,250,None,"02 quartos, sendo 01 suíte com móveis planejad...",Super destaque,1,275000
627,4,Quadra ALC-SO 141 - Mirante do Lago Avenida NS...,408,3,5,3,None,None,Casa no Mirante do Lago 210 m2 de area const...,Super destaque,1,1750000
628,5,"Plano Diretor Sul, Palmas",140,3,3,None,None,None,"Belíssimop Imóvel na 309 Sul, quadra nobre de ...",Super destaque,1,620000


In [19]:
destaque_mask = df['Status'] == 'Destaque'
destaque_df = df.loc[destaque_mask, :]
destaque_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
8,9,"Estrada da Floresta, Floresta Sul",360,None,None,None,None,None,O lote/terreno está localizado no bairro Flore...,Destaque,1,270000
12,13,"Avenida Getúlio Vargas, Centro",250,3,None,2,None,None,None,Destaque,1,500000
13,14,"Estrada do Calafate, Portal da Amazonia",360,None,None,None,None,None,"O terreno fica em uma localização excelente, n...",Destaque,1,260000
15,16,"Rua Comércio, Conjunto Manoel Juliao",50,2,1,1,None,None,"Apartamento fica no 4° andar, pronto para fina...",Destaque,1,125000
18,19,"Rua Porto Velho, Loteamento Joafra",250,2,2,2,None,None,"VEND0 casa localizada no loteamento Joafra, pr...",Destaque,1,270000
...,...,...,...,...,...,...,...,...,...,...,...,...
631,8,"Rua 1, Centro Taquaralto",130,3,3,4,None,None,"Casa térrea muito ampla com 3/4 sendo 1 suíte,...",Destaque,1,190000
639,16,"Quadra ARNE 14 Alameda 10, Plano Diretor Norte",87,3,2,2,450,900,"Vista maravilhosa para o lago de palmas, condo...",Destaque,1,430000
642,19,"Plano Diretor Sul, Palmas",130,3,2,2,None,None,Casa Semi-acabada na 1503 Sul - Palmas/TO Voc...,Destaque,1,210000
643,20,"ARSO 53 Alameda 13, Plano Diretor Sul",238,3,3,2,None,None,"QUADRA: 507 SUL TERRENO: 238,3 ÁREA CONSTRUÍDA...",Destaque,1,645000


In [20]:
comum_mask = df['Status'] == 'Comum'
comum_df = df.loc[comum_mask, :]
comum_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
3,4,"Rua C, LBA Vila Betel ll",336,2,2,1,None,None,?Casa na Vila Betel ! Vende-se uma casa no co...,Comum,1,185000
4,5,"Rua Abacabeira, Vila da Amizade",243,3,1,2,None,None,A casa está localizado no bairro Vila da Amiza...,Comum,1,380000
5,6,"Estrada do Calafate, Calafate",50,2,1,1,None,None,Vendo esse imóvel em condomínio fechado. Apart...,Comum,1,89000
6,7,"Rua Belo Horizonte, Ivete Vargas",170,4,2,1,None,None,Vende-se uma casa no Ivete Vargas. A casa poss...,Comum,1,245000
7,8,"Rua São Mateus, Residencial Araca",49,2,1,1,230,300,O apartamento no bairro Residencial Araçá com ...,Comum,1,185000
...,...,...,...,...,...,...,...,...,...,...,...,...
640,17,"Rua P 3, Setor Sul Taquaralto",400,4,3,None,None,None,Fazenda São João Palmas TO 23alq 15alq pasto...,Comum,1,160000
641,18,"Plano Diretor Sul, Palmas",133,3,2,None,None,None,Casa ainda no projeto. Não concluída. 3 quarto...,Comum,1,320000
644,21,"ARSE 23 Alameda 1, Plano Diretor Sul",126,3,4,2,None,None,O apartamento está localizado no bairro Plano ...,Comum,1,650000
646,23,"Rua 31, Jardim Aureny III",140,3,2,4,None,600,A casa no bairro Jardim Aureny III com 140 met...,Comum,1,220000


In [21]:
construcao_mask = df['Status'] == 'Em construção'
construcao_df = df.loc[construcao_mask, :]
construcao_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
26,3,"Rua Valdo Omena, Ponta Verde",37 - 79,1 - 2,1,1 - 2,None,None,Venha conhecer o Smart Stay.,Em construção,1,
98,3,"Rua Parambu, Santa Teresa",58,2,2,1,None,None,Residencial Santorini – BRF Realizações Jardi...,Em construção,1,395000
121,2,"Rua Gonçalves Ledo, Centro",100,3,4,3,None,None,Venha morar onde o melhor da vida acontece! R...,Em construção,1,759000
122,3,"Rua Gonçalves Ledo, Centro",100,3,4,2,None,None,LAZER COMPLETO Piscina adulto e infantil com d...,Em construção,1,
145,2,"SQNW 111 Bloco E, Setor Noroeste",301,4,6,4,None,None,Apartamentos e Coberturas Duplex - 4 quartos -...,Em construção,1,4534606
192,1,"Rua 22, Setor Oeste",219,4,6,4,None,None,Adagio - Opção de 3 suítes com possibilidade d...,Em construção,1,1684727
193,2,"Rua 22, Setor Oeste",88,2 - 3,3,2,None,None,Allegro -Opção de transformar em 3 quartos; Me...,Em construção,1,651407
194,3,"Rua 22, Setor Oeste",148,3 - 4,5,3,None,None,Moderato - Sala com mega esquadria e balcão qu...,Em construção,1,1088094
240,1,"Avenida Ipiranga, Porto",35 - 44,1 - 2,1,1,None,None,"Apartamentos de 2 quartos no Porto, em condomí...",Em construção,1,217990
241,2,"Avenida Fernando Correa da Costa, Jardim Petró...",165,None,5,3,None,None,O Harissa é um empreendimento único em Cuiabá....,Em construção,1,


In [22]:
planta_mask = df['Status'] == 'Na planta'
planta_df = df.loc[planta_mask, :]
planta_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
24,1,"Avenida Comendador Gustavo Paiva, Mangabeiras",39 - 71,None,2,None,None,None,THE ICON PARK OFFICE Um EMPRESARIAL ao lado d...,Na planta,1,297394
74,3,"Rua Fulvio Pennanchi, Planalto",38 - 43,1 - 2,1,1,None,None,"Lazer equipado, Previsão para medição de água ...",Na planta,1,176490
97,2,"Rua Doutor Genésio Salles, Vila Laura",61 - 89,2 - 3,2 - 3,1 - 2,None,None,O melhor da Vila Laura para você. Escolha mais...,Na planta,1,387480
120,1,"Rua Nogueira Acioli, Joaquim Tavora",51 - 65,2 - 3,2 - 3,1 - 2,None,None,"Modernidade, conforto e acessibilidade. Elemen...",Na planta,1,
144,1,"SQNW 302 Bloco H, Setor Noroeste",71,2,3,1,None,None,"Apartamento 2 quartos, com até 2 suítes Noroes...",Na planta,1,1080181
146,3,"SQNW 109 Bloco G, Setor Noroeste",118 - 230,3,2 - 4,2 - 3,None,None,Reserva Mykonos - Apartamentos e Coberturas Du...,Na planta,1,1702889
168,1,"Rua das Marrecas, Centro",32 - 66,1,1,None,None,None,Hub Coliving. Studios com área de convivência ...,Na planta,1,
170,3,"Rua Irineu Marinho, Centro",35,1 - 2,1,None,None,None,Cores do Rio Centro da Cidade RJ: É o primeiro...,Na planta,1,272000
216,1,"Avenida Dom José Delgado, Alemanha",39 - 42,2,1,1,None,None,"Apartamentos de 2 quartos no Alemanha, em cond...",Na planta,1,163990
218,3,"Avenida dos Holandeses, Ponta D'areia",81 - 90,3,1,2,None,None,Imagine alcançar outro nível na sua qualidade ...,Na planta,1,889300


In [23]:
pronto_mask = df['Status'] == 'Pronto para morar'
pronto_df = df.loc[pronto_mask, :]
pronto_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
25,2,"Rua Doutora Rosa Cabús, Jatiúca",40 - 59,1 - 2,1 - 2,1,None,None,"Viva o presente, invista no futuro! Viver é g...",Pronto para morar,1,
72,1,"Avenida Coronel Teixeira, Ponta Negra",66 - 88,2 - 3,2 - 3,1,None,None,Pronto para morar ao lado do Shopping Ponta Ne...,Pronto para morar,1,340000
73,2,"Avenida Coronel Teixeira, Ponta Negra",36 - 49,1 - 2,1,1,None,None,A região da Ponta Negra é um dos pontos mais n...,Pronto para morar,1,
96,1,"3ª Travessa Paripiranga, Bairro da Paz",53 - 71,2 - 3,2,1,None,None,Pronto para morar pertinho da praia!Pronto par...,Pronto para morar,1,
169,2,"Rua Ubaldino do Amaral, Centro",38 - 69,1 - 2,1 - 2,1,None,None,"A vida prática e moderna, como nas cidades mai...",Pronto para morar,1,349900
217,2,"Avenida da História, Cohafuma",28 - 38,1,1,1,None,None,residencial cohafuma: do tamanho certo para vo...,Pronto para morar,1,145000
290,3,"Avenida Álvares Cabral, Lourdes",78 - 83,3,3,2 - 3,None,None,"Lumière Residence Service, o reflexo dos seus ...",Pronto para morar,1,1149572
313,2,"Rua Timbiras, Jurunas",82,3,2,1,None,None,Venha conhecer os diferenciais incríveis desse...,Pronto para morar,1,
362,3,"Avenida Marechal Floriano Peixoto, Centro",80,None,None,None,None,None,Na esquina da Rua Marechal Floriano com Av. Se...,Pronto para morar,1,526316
386,3,"Rua Capitão José da Luz, Coelhos",34 - 62,None,1 - 2,1 - 2,None,None,"Centro empresarial de Recife, o empresarial Jo...",Pronto para morar,1,150000
